
<h1> DS200A Computer Vision Assignment</h1>

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd

<h2>  Part Three: Classifier training and performance assessment. </h2>	

In [2]:
def split(df):
    # Split the data into a training set, and test set 
    y = data[["Encodings"]]
    X = data.drop(["Encodings", "Pictures"], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
    return X_train, X_test, y_train, y_test


def accuracy(pred, actual):
    # Calculate the accuracy percentage of the predicted values
    return sum(pred == actual) / len(actual)

<h3>  Train models using all of the following methods below. Be sure to drop the actual image column, and the encoding</h3>	Take note of the differences in accuracy, and methods.


Let's load the dataset first.

In [3]:
data = pd.read_pickle("train_with_feature.pkl")
data.head()

Pictures  Encodings    size  \
0  [[[255, 255, 255], [255, 255, 255], [255, 255,...          0  194535   
1  [[[88, 56, 17], [95, 63, 24], [101, 69, 30], [...          0  208278   
2  [[[255, 255, 255], [255, 255, 255], [255, 255,...          0  201096   
3  [[[255, 255, 255], [255, 255, 255], [255, 255,...          0  221352   
4  [[[255, 255, 255], [255, 255, 255], [255, 255,...          0  166662   

        avg_r  aspect_ratio       avg_b       avg_g    avg_hue     avg_sat  \
0  170.123787      2.381818  111.753798  147.788681  42.172087  120.420202   
1   98.426973      2.293103   67.531631  104.499870  40.144197  126.898583   
2  174.865318      2.292398  163.577814  175.592150  37.242541   50.880937   
3  210.739822      2.179348  164.805242  189.280101  23.024612   69.222569   
4  147.548565      2.794326  127.917054  155.967905  44.592073   89.481027   

   avg_bright ...   190  191   192  193  194  195  196  197  198  199  
0  178.581633 ...   1.0  5.0   6.0  1.0  1.0  1.0  0.0  1.0  2.0  0.0  
1  110.955982 ...   0.0  9.0   2.0  1.0  1.0  0.0  2.0  0.0  1.0  0.0  
2  179.086287 ...   0.0  2.0  10.0  0.0  2.0  0.0  0.0  0.0  1.0  0.0  
3  210.805961 ...   1.0  0.0  11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  157.456385 ...   1.0  5.0   5.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  

[5 rows x 410 columns]

In [4]:
X_train, X_test, y_train, y_test = split(data)

Logistic Regression

In [5]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42, 
                         multi_class='auto',
                         solver='liblinear',
                         max_iter=5000)
print("CV =", np.mean(cross_val_score(clf, X_train, y_train.values.ravel(), cv=5)))

CV = 0.42691618162952844


In [6]:
test_acc = accuracy(clf.fit(X_train, y_train.values.ravel()).predict(X_test), y_test.values.ravel())
print("test = ", test_acc)

test =  0.4469026548672566


K-nearest Neighbors

In [7]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=120)

print("CV =", np.mean(cross_val_score(neigh, X_train, y_train.values.ravel(), cv=5)))

CV = 0.21333623879961316


In [8]:
test_acc = accuracy(neigh.fit(X_train, y_train.values.ravel()).predict(X_test), y_test.values.ravel())
print("test = ", test_acc)

test =  0.20353982300884957


Classification Tree

In [9]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=72)
print("CV =", np.mean(cross_val_score(clf, X_train, y_train, cv=5)))

CV = 0.32792199963361907


In [10]:
test_acc = accuracy(clf.fit(X_train, y_train.values.ravel()).predict(X_test), y_test.values.ravel())
print("test = ", test_acc)

test =  0.336283185840708


Random Forest

In [11]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_depth=18, random_state=42)
print("CV =", np.mean(cross_val_score(clf, X_train, y_train.values.ravel(), cv=5)))

CV = 0.4424022311689285


In [12]:
test_acc = accuracy(clf.fit(X_train, y_train.values.ravel()).predict(X_test), y_test.values.ravel())
print("test = ", test_acc)

test =  0.47345132743362833


Support Vector Machine

In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MinMaxScaler

clf = OneVsRestClassifier(SVC(gamma="auto", probability=False, C=400))
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train_scaled = scaling.transform(X_train)
print("CV =", np.mean(cross_val_score(clf, X_train_scaled, y_train.values.ravel(), cv=5)))

CV = 0.4756420974526948


In [14]:
X_test_scaled = scaling.transform(X_test)
test_acc = accuracy(clf.fit(X_train_scaled, y_train.values.ravel()).predict(X_test_scaled), y_test.values.ravel())
print("test = ", test_acc)

test =  0.5132743362831859


Support Vector Machine gives the best results. Let's use that for our final prediction!

In [20]:
data = pd.read_pickle("train_with_feature.pkl")
y = data[["Encodings"]]
X = data.drop(["Encodings", "Pictures"], axis=1)
scaling = MinMaxScaler(feature_range=(-1, 1)).fit(X)
X_scaled = scaling.transform(X)
clf = OneVsRestClassifier(SVC(gamma="auto", probability=False, C=400))
clf.fit(X_scaled, y.values.ravel())
X_test = pd.read_pickle("test_with_feature.pkl").drop(["Pictures"], axis=1)
X_test_scaled = scaling.transform(X_test)
pred = clf.predict(X_test_scaled)

In [21]:
np.savetxt("predictions.csv", pred, delimiter=",")